In [25]:
import time, threading, queue, sys, gc
from typing import Tuple

import numpy as np
import pyqtgraph as pg
from PyQt6 import QtCore

In [26]:
# ------------------------------------------------------------  parameters
DEBUG           = True           # set True for jitter log
SAMPLE_RATE     = 100_000         # Hz
CHUNK_SEC       = 0.2             # s
N_SAMPLES       = int(SAMPLE_RATE * CHUNK_SEC)
QUEUE_DEPTH     = 4               # raw AB backlog  (≈0.8 s)
QUAD_DEPTH      = 4               # processed backlog (same)
RUN_SEC         = 30              # auto‑stop after 10 s
DISPLAY_SEC       = 0.02            # *** fixed x-axis window width (s) ***
GUI_INTERVAL_MS   = 50             # *** refresh every 50 ms (≈20 FPS) ***

PULSE_HEIGHT    = 5.0             # amplitude
PULSE_WIDTH     = 0.002           # period  (s)
PULSE_DUTY      = 0.5             # duty
PULSE_PHASE_A   = 0.0             # phase offset (s)
PULSE_PHASE_B   = 0.0005          # phase offset (s)

QUADPULSE_WIDTH = 0.00025         # width (s)
THRESHOLD_DEFAULT = 2.5           # logic threshold (V)



# ------------------------------------------------------------  queues & stop flag
buf_q  = queue.Queue(maxsize=QUEUE_DEPTH)   # raw (t, A, B)
quad_q = queue.Queue(maxsize=QUAD_DEPTH)    # processed (t, A, B, quad)
stop_ev = threading.Event()

In [ ]:
def gen_chunk_pulse(
    t: np.ndarray,
    *,
    height: float = PULSE_HEIGHT,
    width: float = PULSE_WIDTH,
    duty: float = PULSE_DUTY,
    phase: float = 0.0,
) -> np.ndarray:
    mod = (t + phase) % width
    return np.where(mod < duty * width, height, 0.0).astype(np.float32)


REL_AXIS = np.arange(N_SAMPLES, dtype=np.float32) / SAMPLE_RATE  # 0 … 0.199 999 s

# ------------------------------------------------------------  AB → direction → quad helpers


def gen_pulse_direction(
    dA: np.ndarray, dB: np.ndarray, *, threshold: float
) -> np.ndarray:
    A = dA > threshold
    B = dB > threshold
    A_prev = np.concatenate(([A[0]], A[:-1]))
    B_prev = np.concatenate(([B[0]], B[:-1]))
    return ((B_prev ^ A).astype(int) - (A_prev ^ B).astype(int)).astype(np.int8)


def pulse_count(dir_log: np.ndarray) -> int:
    return int(np.sum(dir_log))


def gen_quad_pulse(
    t: np.ndarray, dir_log: np.ndarray, width: float, height: float, sampling_rate: int
) -> np.ndarray:
    samples = int(width * sampling_rate)
    if samples <= 0:
        return np.zeros_like(dir_log, dtype=np.float32)
    base = np.full(samples, height, dtype=np.float32)
    return np.convolve(dir_log, base, mode="full")[: len(t)]


# ------------------------------------------------------------  producer thread


def generator() -> None:
    """Generate AB rectangular‑wave chunks at real‑time cadence.
    Drops the newest chunk if raw queue is already full (latency > 0.8 s).
    """
    chunk_idx = 0
    next_t = time.perf_counter()
    while not stop_ev.is_set():
        base = chunk_idx * CHUNK_SEC
        t_axis = REL_AXIS + base
        pulse_A = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_A)
        pulse_B = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_B)

        try:
            buf_q.put_nowait((t_axis, pulse_A, pulse_B))
        except queue.Full:
            pass  # drop chunk

        chunk_idx += 1
        next_t += CHUNK_SEC
        sleep = next_t - time.perf_counter()
        if sleep > 0:
            time.sleep(sleep)
        else:
            next_t = time.perf_counter()


# ------------------------------------------------------------  consumer thread


def process_chunk(
    t: np.ndarray, dA: np.ndarray, dB: np.ndarray
) -> Tuple[int, np.ndarray]:
    dir_log = gen_pulse_direction(dA, dB, threshold=THRESHOLD_DEFAULT)
    return pulse_count(dir_log), gen_quad_pulse(
        t, dir_log, QUADPULSE_WIDTH, PULSE_HEIGHT, SAMPLE_RATE
    )


def consumer() -> None:
    last_ts = time.perf_counter()
    cum_count = 0
    while not stop_ev.is_set():
        try:
            t, pulse_A, pulse_B = buf_q.get(timeout=0.5)
        except queue.Empty:
            continue
        now = time.perf_counter()
        count, quad_sig = process_chunk(t, pulse_A, pulse_B)
        cum_count += count
        velocity = count / CHUNK_SEC / 2048  # counts per second

        if DEBUG:
            jitter = (now - last_ts) * 1e3
            print(
                f"EPOCH = {now:.2f} jitter = {jitter:6.2f} ms  Δc={count:+d}  c={cum_count}  v={velocity:.3f}"
            )
        last_ts = now

        try:
            quad_q.put_nowait(
                (t, pulse_A, pulse_B, quad_sig, t[-1], cum_count, velocity)
            )
        except queue.Full:
            pass

        del t, pulse_A, pulse_B, quad_sig
        # gc.collect() # uncomment to force garbage collection
        buf_q.task_done()


# ------------------------------------------------------------  GUI
HISTORY = int(SAMPLE_RATE * DISPLAY_SEC)
COUNT_HISTORY = int(RUN_SEC / CHUNK_SEC) * 6
VELO_HISTORY = COUNT_HISTORY


def start_gui() -> None:
    pg.setConfigOptions(useOpenGL=True, background="w", foreground="k")
    app = pg.mkQApp("Live plots (10 s)")

    win = pg.GraphicsLayoutWidget(title="AB / Quad / Count / Velocity")
    win.show()

    # [0, 0] A/B -------------------------------------------------------
    plt_ab = win.addPlot(row=0, col=0, title="RAW A / B")
    curve_A = plt_ab.plot(pen=pg.mkPen("#ff4b00", width=3))
    curve_B = plt_ab.plot(pen=pg.mkPen("#005aff", width=3))
    plt_ab.setLabel(axis="left", text="Amplitude (V)")
    plt_ab.setLabel(axis="bottom", text="Time [s]")
    plt_ab.setYRange(-0.5, PULSE_HEIGHT + 0.5)

    # [1,0] Quad waveform --------------------------------------------
    plt_q = win.addPlot(row=1, col=0, title="Quad pulse")
    curve_Q = plt_q.plot(pen=pg.mkPen("m", width=3))
    plt_q.setLabel(axis="left", text="Amplitude (V)")
    plt_q.setLabel(axis="bottom", text="Time [s]")
    plt_q.setYRange(-0.5 - PULSE_HEIGHT, PULSE_HEIGHT + 0.5)

    # [0,1] count (fixed x‑axis) -------------------------------------------
    plt_cnt = win.addPlot(row=0, col=1, title="Quad count")
    curve_cnt = plt_cnt.plot(pen=pg.mkPen("#03af7a", width=3))
    plt_cnt.setXRange(0, RUN_SEC, padding=0)
    plt_cnt.enableAutoRange("x", False)
    plt_cnt.setLabel("left", "Count")
    plt_cnt.setLabel("bottom", "Time [s]")

    # [1,1] velocity (fixed x‑axis) ----------------------------------------
    plt_vel = win.addPlot(row=1, col=1, title="Velocity (counts/s)")
    curve_vel = plt_vel.plot(pen=pg.mkPen("#00a0e9", width=3))
    plt_vel.setXRange(0, RUN_SEC, padding=0)
    plt_vel.enableAutoRange("x", False)
    plt_vel.setLabel("left", "Velocity [c/s]")
    plt_vel.setLabel("bottom", "Time [s]")

    # plt_q.setXLink(plt_ab)      # 上段 2 枚をリンク
    # plt_vel.setXLink(plt_cnt)   # 下段 2 枚をリンク

    # buffers ---------------------------------------------------------
    xs = ya = yb = yq = np.empty(0, dtype=np.float32)
    xs_cnt = y_cnt = np.empty(0, dtype=np.float32)
    xs_vel = y_vel = np.empty(0, dtype=np.float32)

    def refresh():
        nonlocal xs, ya, yb, yq, xs_cnt, y_cnt, xs_vel, y_vel
        try:
            while True:
                t_ax, pA, pB, qsig, t_end, cum_cnt, vel = quad_q.get_nowait()
                xs = np.concatenate((xs, t_ax))[-HISTORY:]
                ya = np.concatenate((ya, pA))[-HISTORY:]
                yb = np.concatenate((yb, pB))[-HISTORY:]
                yq = np.concatenate((yq, qsig))[-HISTORY:]

                xs_cnt = np.append(xs_cnt, t_end)[-COUNT_HISTORY:]
                y_cnt = np.append(y_cnt, cum_cnt)[-COUNT_HISTORY:]

                xs_vel = np.append(xs_vel, t_end)[-VELO_HISTORY:]
                y_vel = np.append(y_vel, vel)[-VELO_HISTORY:]
                quad_q.task_done()
        except queue.Empty:
            pass

        # scrolling window for waveforms only
        if xs.size:
            start = xs[-1] - DISPLAY_SEC
            plt_ab.setXRange(start, xs[-1], padding=0)
            plt_q.setXRange(start, xs[-1], padding=0)

        # --- push data to curves ---
        curve_A.setData(xs, ya)
        curve_B.setData(xs, yb)
        curve_Q.setData(xs, yq)
        curve_cnt.setData(xs_cnt, y_cnt)
        curve_vel.setData(xs_vel, y_vel)

    timer = QtCore.QTimer()
    timer.timeout.connect(refresh)
    timer.start(GUI_INTERVAL_MS)  # ≈20 FPS

    # auto‑stop after RUN_SEC
    QtCore.QTimer.singleShot(RUN_SEC * 1000, lambda: (stop_ev.set(), app.quit()))
    app.exec()

In [ ]:
gen_th = threading.Thread(target=generator, daemon=True)
con_th = threading.Thread(target=consumer, daemon=True)

gen_th.start()
con_th.start()

start_gui()

# join threads and exit
stop_ev.set()
gen_th.join()
con_th.join()

print("Graceful shutdown.")

EPOCH = 190360.86 jitter = 171.85 ms  Δc=-399  c=-399  v=-0.974
EPOCH = 190360.90 jitter =  32.79 ms  Δc=-399  c=-798  v=-0.974
EPOCH = 190361.13 jitter = 231.42 ms  Δc=-399  c=-1197  v=-0.974
EPOCH = 190361.32 jitter = 195.63 ms  Δc=-399  c=-1596  v=-0.974
EPOCH = 190361.51 jitter = 190.99 ms  Δc=-399  c=-1995  v=-0.974
EPOCH = 190361.72 jitter = 205.64 ms  Δc=-400  c=-2395  v=-0.977
EPOCH = 190361.89 jitter = 174.80 ms  Δc=-399  c=-2794  v=-0.974
EPOCH = 190362.10 jitter = 200.53 ms  Δc=-399  c=-3193  v=-0.974
EPOCH = 190362.31 jitter = 218.10 ms  Δc=-399  c=-3592  v=-0.974
EPOCH = 190362.51 jitter = 198.50 ms  Δc=-399  c=-3991  v=-0.974
EPOCH = 190362.72 jitter = 206.04 ms  Δc=-400  c=-4391  v=-0.977
EPOCH = 190362.91 jitter = 187.79 ms  Δc=-399  c=-4790  v=-0.974
EPOCH = 190363.10 jitter = 198.88 ms  Δc=-399  c=-5189  v=-0.974
EPOCH = 190363.30 jitter = 190.50 ms  Δc=-399  c=-5588  v=-0.974
EPOCH = 190363.50 jitter = 202.89 ms  Δc=-399  c=-5987  v=-0.974
EPOCH = 190363.70 jitter = 